In [1]:
import pickle
import numpy as np

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models, losses, optimizers, Model
import numpy as np

2024-11-29 17:15:10.642835: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-29 17:15:10.693800: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-29 17:15:10.693852: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-29 17:15:10.693891: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-29 17:15:10.705442: I tensorflow/core/platform/cpu_feature_g

In [5]:
pickle_MCI_Features_file_path = r'./feature_extraction/MCI_final_features_all_1.pkl'
with open(pickle_MCI_Features_file_path, 'rb') as file:
    MCI_funct_features = pickle.load(file)
MCI_funct_features.shape
MCI_funct_features = np.array(MCI_funct_features)

In [6]:
MCI_funct_features.shape

(53, 13456)

In [7]:
pickle_MCI_Features_file_path = r'feature_extraction/cat12_structural_features.pkl'
with open(pickle_MCI_Features_file_path, 'rb') as file:
    MCI_Struct_features = pickle.load(file)
MCI_Struct_features.shape
MCI_Struct_features = np.array(MCI_Struct_features)

In [8]:
MCI_Struct_features.shape

(53, 169, 205, 169)

In [9]:
# Define the same file path
file_path = r'feature_extraction/Structal_cat_fpickle.pkl'

# Load the NumPy array from the file
with open(file_path, "rb") as file:  # 'rb' means read binary
    loaded_structural_features = pickle.load(file)

print("structural_features loaded successfully!")
print("Shape of loaded array:", loaded_structural_features.shape)


structural_features loaded successfully!
Shape of loaded array: (40, 169, 205, 169)


In [10]:
pickle_AD_Features_file_path = r"feature_extraction/func_features_AD.pkl"
with open(pickle_AD_Features_file_path, 'rb') as file:
    AD_funct_features = pickle.load(file)
AD_funct_features = np.array(AD_funct_features)
AD_funct_features.shape

(40, 13456)

In [11]:
MCI_funct_features.shape

(53, 13456)

In [12]:
MCI_Struct_features.shape


(53, 169, 205, 169)

In [13]:
AD_funct_features.shape


(40, 13456)

In [14]:
loaded_structural_features.shape

(40, 169, 205, 169)

In [15]:

# Data Shapes
AD_Struct_features = loaded_structural_features
# Labels
MCI_labels = np.zeros(53)  # Label 0 for MCI
AD_labels = np.ones(40)    # Label 1 for AD

# Combine functional and structural features
funct_features = np.vstack((MCI_funct_features, AD_funct_features))
struct_features = np.vstack((MCI_Struct_features, AD_Struct_features))
labels = np.hstack((MCI_labels, AD_labels))

In [16]:
import numpy as np

# Assuming you have loaded your SMRI data into variables
# MCI_Struct_features and AD_Struct_features

# Combine MCI and AD structural features
SMRI_data = np.concatenate([MCI_Struct_features, AD_Struct_features], axis=0)
SMRI_labels = np.concatenate([
    np.zeros(len(MCI_Struct_features)),  # Label 0 for MCI
    np.ones(len(AD_Struct_features))     # Label 1 for AD
], axis=0)

# Add a channel dimension
SMRI_data = np.expand_dims(SMRI_data, axis=-1)  # Shape becomes (num_samples, 169, 205, 169, 1)

In [17]:
# Assuming you have loaded your fMRI data into variables
# MCI_funct_features and AD_funct_features

# Combine MCI and AD functional features
fMRI_data = np.concatenate([MCI_funct_features, AD_funct_features], axis=0)

# Reshape and add channel dimension
fMRI_data = fMRI_data.reshape(-1, 116, 116)
fMRI_data = np.expand_dims(fMRI_data, axis=-1)  # Shape becomes (num_samples, 116, 116, 1)

In [18]:
labels = np.concatenate([
    np.zeros(len(MCI_funct_features)),  # Label 0 for MCI
    np.ones(len(AD_funct_features))     # Label 1 for AD
], axis=0)

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
# Split data into training and testing sets
X_train_3d, X_test_3d, X_train_2d, X_test_2d, y_train, y_test = train_test_split(
    SMRI_data,
    fMRI_data,
    labels,
    test_size=0.2,
    random_state=42
)

In [21]:
# Discriminator labels (real images are labeled as 1)
real_labels_train = np.ones((X_train_3d.shape[0], 1))
real_labels_test = np.ones((X_test_3d.shape[0], 1))

# For simplicity, use real labels for reconstructed images
disc_targets_3d_train = real_labels_train
disc_targets_2d_train = real_labels_train

disc_targets_3d_test = real_labels_test
disc_targets_2d_test = real_labels_test

In [22]:
import tensorflow as tf
from tensorflow.keras import layers, Model, Input

def build_encoder_3d(input_shape):
    inputs = Input(shape=input_shape) # 169x205x169x1 (input, 1 channel)
    x = layers.Conv3D(32, (3, 3, 3), activation='gelu', padding='valid')(inputs) # 166×203×166 x32
    x = layers.MaxPooling3D((2, 2, 2))(x) # 83×101×83 x32
    x = layers.Conv3D(64, (3, 3, 3), activation='gelu', padding='valid')(x) # 81×99×81 x64
    x = layers.MaxPooling3D((2, 2, 2))(x) # 40×49×40 x64
    x = layers.Conv3D(128, (3, 3, 3), activation='gelu', padding='valid')(x) # 38×47×38 128
    x = layers.Flatten()(x)
    latent = layers.Dense(256, activation='gelu')(x)
    encoder = Model(inputs, latent, name='encoder_3d')
    return encoder

In [23]:
def build_encoder_2d(input_shape):
    inputs = Input(shape=input_shape) # 116x116 x1
    x = layers.Conv2D(32, (3, 3), activation='gelu', padding='valid')(inputs) # 114x114 x32
    x = layers.MaxPooling2D((2, 2))(x) # 57x57 x32
    x = layers.Conv2D(64, (3, 3), activation='gelu', padding='valid')(x) # 55x55 x64
    x = layers.MaxPooling2D((2, 2))(x) # 27x27 x64
    x = layers.Conv2D(128, (3, 3), activation='gelu', padding='valid')(x) # 25x25 x128
    x = layers.Flatten()(x)
    latent = layers.Dense(256, activation='gelu')(x)
    encoder = Model(inputs, latent, name='encoder_2d')
    return encoder

In [24]:
def build_decoder_3d(latent_dim, output_shape):
    latent_inputs = Input(shape=(latent_dim,))
    reshape_shape = np.array([output_shape[0]//8, output_shape[1]//8, output_shape[2]//8, 128])
    x = layers.Dense(np.prod(reshape_shape), activation='gelu')(latent_inputs)
    x = layers.Reshape(reshape_shape)(x)
    x = layers.Conv3DTranspose(128, (3, 3, 3), strides=2, padding='valid', activation='gelu')(x)
    x = layers.Conv3DTranspose(64, (3, 3, 3), strides=2, padding='valid', activation='gelu')(x)
    x = layers.Conv3DTranspose(32, (3, 3, 3), strides=2, padding='valid', activation='gelu')(x)
    # Adjust the final Conv3DTranspose layer to match the expected output shape
    x = layers.Conv3DTranspose(1, (3, 3, 3), strides=(1, 1, 1), padding='valid', activation='sigmoid')(x)
    # x = layers.ZeroPadding3D(padding=((0, 1), (0, 5), (0, 1)))(x)  # Add padding to match (169, 205, 169)    
    x = layers.Cropping3D(cropping=((4, 4), (2, 2), (4, 4)))(x)
    outputs = x
    
    # Ensure the output shape matches the input shape of the discriminator
    assert outputs.shape[1:] == output_shape, f"Output shape {outputs.shape[1:]} does not match expected shape {output_shape}"
    
    decoder = Model(latent_inputs, outputs, name='decoder_3d')
    return decoder

In [25]:
def build_decoder_2d(latent_dim, output_shape):
    latent_inputs = Input(shape=(latent_dim,))
    x = layers.Dense((output_shape[0]//4)*(output_shape[1]//4)*64, activation='gelu')(latent_inputs)
    x = layers.Reshape((output_shape[0]//4, output_shape[1]//4, 64))(x)
    x = layers.Conv2DTranspose(128, (3, 3), strides=2, padding='valid', activation='gelu')(x)
    x = layers.Conv2DTranspose(64, (3, 3), strides=2, padding='valid', activation='gelu')(x)
    x = layers.Conv2DTranspose(1, (3, 3), padding='valid', activation='sigmoid')(x)
    outputs = layers.Cropping2D(cropping=((2, 3), (2, 3)))(x) 

    assert outputs.shape[1:-1] == output_shape, f"Output shape {outputs.shape[1:-1]} does not match expected shape {output_shape}"


    decoder = Model(latent_inputs, outputs, name='decoder_2d')
    return decoder

In [26]:
def build_discriminator_3d(input_shape):
    inputs = Input(shape=input_shape)
    x = layers.Conv3D(32, (3, 3, 3), activation='gelu')(inputs)
    x = layers.MaxPooling3D((2, 2, 2))(x)
    x = layers.Conv3D(64, (3, 3, 3), activation='gelu')(x)
    x = layers.Flatten()(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    discriminator = Model(inputs, outputs, name='discriminator_3d')
    return discriminator

In [27]:
def build_discriminator_2d(input_shape):
    inputs = Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='gelu')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='gelu')(x)
    x = layers.Flatten()(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    discriminator = Model(inputs, outputs, name='discriminator_2d')
    return discriminator

In [28]:
def build_classifier(input_dim, num_classes):
    inputs = Input(shape=(input_dim,))
    x = layers.Dense(256, activation='gelu')(inputs)
    x = layers.Dense(128, activation='gelu')(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    classifier = Model(inputs, outputs, name='classifier')
    return classifier

In [29]:
# Input shapes
input_shape_3d = (169, 205, 169, 1)
input_shape_2d = (116, 116, 1)
latent_dim = 256
num_classes = 2

# Build models
encoder_3d = build_encoder_3d(input_shape_3d)
encoder_2d = build_encoder_2d(input_shape_2d)
decoder_3d = build_decoder_3d(latent_dim, input_shape_3d)
decoder_2d = build_decoder_2d(latent_dim, input_shape_2d[:-1])
discriminator_3d = build_discriminator_3d(input_shape_3d)
discriminator_2d = build_discriminator_2d(input_shape_2d)
# Build classifier that accepts concatenated latent vectors
classifier = build_classifier(latent_dim * 2, num_classes)

# Inputs
input_3d = Input(shape=input_shape_3d)
input_2d = Input(shape=input_shape_2d)

# Encoding
latent_3d = encoder_3d(input_3d)
latent_2d = encoder_2d(input_2d)

# Concatenate the two latent representations
combined_latent = layers.Concatenate()([latent_3d, latent_2d])

# Decoding
reconstructed_3d = decoder_3d(latent_3d)
reconstructed_2d = decoder_2d(latent_2d)

# Discriminator outputs
disc_output_3d = discriminator_3d(reconstructed_3d)
disc_output_2d = discriminator_2d(reconstructed_2d)

# Get classification output
classification_output = classifier(combined_latent)

# Define the combined model
model = Model(
    inputs=[input_3d, input_2d],
    outputs=[
        reconstructed_3d,
        reconstructed_2d,
        disc_output_3d,
        disc_output_2d,
        classification_output
    ]
)

In [31]:
# Loss weights
lambda_reconstruction = 1.0
lambda_adversarial = 0.1
lambda_classification = 1.0

# Compile the model
model.compile(
    optimizer='adam',
    loss={
        'decoder_3d': 'mse',
        'decoder_2d': 'mse',
        'discriminator_3d': tf.keras.losses.BinaryCrossentropy(),
        'discriminator_2d': tf.keras.losses.BinaryCrossentropy(),
        'classifier': 'sparse_categorical_crossentropy'
    },
    loss_weights={
        'decoder_3d': lambda_reconstruction,
        'decoder_2d': lambda_reconstruction,
        'discriminator_3d': lambda_adversarial,
        'discriminator_2d': lambda_adversarial,
        'classifier': lambda_classification
    },
    metrics={
        'classifier': 'accuracy'
    }
)

In [32]:
# Discriminator labels (real images are labeled as 1)
real_labels = np.ones((SMRI_data.shape[0], 1))
fake_labels = np.zeros((SMRI_data.shape[0], 1))

# For simplicity, use real labels for reconstructed images (you can adjust as needed)
disc_targets_3d = real_labels
disc_targets_2d = real_labels

In [ ]:
# Train the model
history = model.fit(
    [X_train_3d, X_train_2d],
    {
        'decoder_3d': X_train_3d,
        'decoder_2d': X_train_2d,
        'discriminator_3d': disc_targets_3d_train,
        'discriminator_2d': disc_targets_2d_train,
        'classifier': y_train
    },
    epochs=20,
    batch_size=8,
    validation_data=(
        [X_test_3d, X_test_2d],
        {
            'decoder_3d': X_test_3d,
            'decoder_2d': X_test_2d,
            'discriminator_3d': disc_targets_3d_test,
            'discriminator_2d': disc_targets_2d_test,
            'classifier': y_test
        }
    )
)

Epoch 1/20


2024-11-29 17:16:28.129110: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x703e780a5fc0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-11-29 17:16:28.129203: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2024-11-29 17:16:28.196251: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-11-29 17:16:29.506232: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2024-11-29 17:16:29.513654: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2024-11-29 17:17:11.973926: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2024-11-29 17:17:12.01966

 1/10 [==>...........................] - ETA: 11:03 - loss: 1.0900 - decoder_3d_loss: 0.1957 - decoder_2d_loss: 0.0669 - discriminator_3d_loss: 0.6710 - discriminator_2d_loss: 0.6723 - classifier_loss: 0.6932 - classifier_accuracy: 0.5000

2024-11-29 17:17:38.563460: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.


 4/10 [===========>..................] - ETA: 1:54 - loss: nan - decoder_3d_loss: nan - decoder_2d_loss: nan - discriminator_3d_loss: nan - discriminator_2d_loss: nan - classifier_loss: nan - classifier_accuracy: 0.5000                 

2024-11-29 17:18:35.585916: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.


 5/10 [==============>...............] - ETA: 1:34 - loss: nan - decoder_3d_loss: nan - decoder_2d_loss: nan - discriminator_3d_loss: nan - discriminator_2d_loss: nan - classifier_loss: nan - classifier_accuracy: 0.5000

2024-11-29 17:18:53.352648: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.


10/10 [==============================] - 261s 21s/step - loss: nan - decoder_3d_loss: nan - decoder_2d_loss: nan - discriminator_3d_loss: nan - discriminator_2d_loss: nan - classifier_loss: nan - classifier_accuracy: 0.4865 - val_loss: nan - val_decoder_3d_loss: nan - val_decoder_2d_loss: nan - val_discriminator_3d_loss: nan - val_discriminator_2d_loss: nan - val_classifier_loss: nan - val_classifier_accuracy: 0.6842
Epoch 2/20
 5/10 [==============>...............] - ETA: 1:32 - loss: nan - decoder_3d_loss: nan - decoder_2d_loss: nan - discriminator_3d_loss: nan - discriminator_2d_loss: nan - classifier_loss: nan - classifier_accuracy: 0.6000

In [35]:
# Evaluate the classifier accuracy on the test set
evaluation = model.evaluate(
    [X_test_3d, X_test_2d],
    {
        'decoder_3d': X_test_3d,
        'decoder_2d': X_test_2d,
        'discriminator_3d': disc_targets_3d_test,
        'discriminator_2d': disc_targets_2d_test,
        'classifier': y_test
    },
    verbose=0
)

# Get the index of the classification loss and accuracy (depends on the order of outputs)
classification_accuracy = evaluation[model.metrics_names.index('classifier_accuracy')]

print(f'Classifier Test Accuracy: {classification_accuracy * 100:.2f}%')

Classifier Test Accuracy: 68.42%


In [4]:
devices = tf.config.list_physical_devices()
print(devices)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
tf.config.set_visible_devices([], 'GPU')

In [36]:
model.save("models/fusion_model_1.h5")

/home/tripti/miniconda3/lib/python3.9/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


OSError: [Errno 28] Can't close file (file write failed: time = Thu Nov 28 18:09:47 2024
, filename = 'models/fusion_model_1.h5', file descriptor = 84, errno = 28, error message = 'No space left on device', buf = 0xbfd3120, total write size = 47824, bytes this sub-write = 47824, bytes actually written = 18446744073709551615, offset = 0)